In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

In [6]:
df_train = pd.read_csv('../../../datasets/used_bike/Used_Bikes.csv')

In [7]:
df_train.head()

,bike_name,price,city,kms_driven,owner,age,power,brand
0,TVS Star City Plus Dual Tone 110cc,35000.0,Ahmedabad,17654.0,First Owner,3.0,110.0,TVS
1,Royal Enfield Classic 350cc,119900.0,Delhi,11000.0,First Owner,4.0,350.0,Royal Enfield
2,Triumph Daytona 675R,600000.0,Delhi,110.0,First Owner,8.0,675.0,Triumph
3,TVS Apache RTR 180cc,65000.0,Bangalore,16329.0,First Owner,4.0,180.0,TVS
4,Yamaha FZ S V 2.0 150cc-Ltd. Edition,80000.0,Bangalore,10000.0,First Owner,3.0,150.0,Yamaha


In [22]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32648 entries, 0 to 32647
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bike_name   32648 non-null  object 
 1   price       32648 non-null  float64
 2   city        32648 non-null  object 
 3   kms_driven  32648 non-null  float64
 4   owner       32648 non-null  object 
 5   age         32648 non-null  float64
 6   power       32648 non-null  float64
 7   brand       32648 non-null  object 
dtypes: float64(4), object(4)
memory usage: 2.0+ MB


In [23]:
df_train.dtypes

bike_name      object
price         float64
city           object
kms_driven    float64
owner          object
age           float64
power         float64
brand          object
dtype: object

In [24]:
df_train.shape

(32648, 8)

# feature encoding 

In [ ]:
#feature encoding 

# Assuming your DataFrame is called df and has a 'brand' column

# Clean brand names (remove whitespace)
df_train['brand'] = df_train['brand'].str.strip()

# Create brand category mapping
brand_category_map = {
    # Premium
    'BMW': 'premium',
    'Ducati': 'premium',
    'Harley-Davidson': 'premium',
    'Triumph': 'premium',
    'Indian': 'premium',
    'MV': 'premium',
    'Benelli': 'premium',
    'Kawasaki': 'premium',

    # Mid-range
    'Royal Enfield': 'mid',
    'KTM': 'mid',
    'Jawa': 'mid',
    'Hyosung': 'mid',
    'Yamaha': 'mid',
    'Suzuki': 'mid',

    # Budget
    'Hero': 'budget',
    'Honda': 'budget',
    'TVS': 'budget',
    'Bajaj': 'budget',
    'Mahindra': 'budget',
    'LML': 'budget',
    'Rajdoot': 'budget',
    'Ideal': 'budget'
}

# Apply the mapping
df_train['brand_category'] = df_train['brand'].map(brand_category_map)

#check the result
print(df_train[['brand', 'brand_category']].drop_duplicates().sort_values(by='brand'))


                brand brand_category
359               BMW        premium
12              Bajaj         budget
18            Benelli        premium
92             Ducati        premium
133   Harley-Davidson        premium
7                Hero         budget
6               Honda         budget
110           Hyosung            mid
7467            Ideal         budget
367            Indian        premium
288              Jawa            mid
33                KTM            mid
61           Kawasaki        premium
4960              LML         budget
6792               MV        premium
47           Mahindra         budget
1519          Rajdoot         budget
1       Royal Enfield            mid
17             Suzuki            mid
0                 TVS         budget
2             Triumph        premium
4              Yamaha            mid
5017            Yezdi            NaN


In [9]:
# One-hot encode the 'brand_category' column
df_train_encoded = pd.get_dummies(df_train, columns=['brand_category'], prefix='brand_cat')

# View the result
print(df_train_encoded.head())


                              bike_name     price       city  kms_driven  \
0    TVS Star City Plus Dual Tone 110cc   35000.0  Ahmedabad     17654.0   
1           Royal Enfield Classic 350cc  119900.0      Delhi     11000.0   
2                  Triumph Daytona 675R  600000.0      Delhi       110.0   
3                  TVS Apache RTR 180cc   65000.0  Bangalore     16329.0   
4  Yamaha FZ S V 2.0 150cc-Ltd. Edition   80000.0  Bangalore     10000.0   

         owner  age  power          brand  brand_cat_budget  brand_cat_mid  \
0  First Owner  3.0  110.0            TVS              True          False   
1  First Owner  4.0  350.0  Royal Enfield             False           True   
2  First Owner  8.0  675.0        Triumph             False          False   
3  First Owner  4.0  180.0            TVS              True          False   
4  First Owner  3.0  150.0         Yamaha             False           True   

   brand_cat_premium  
0              False  
1              False  
2    

In [10]:
df_train[df_train['brand'] == 'Royal Enfield'][['bike_name', 'brand_category']]


,bike_name,brand_category
1,Royal Enfield Classic 350cc,mid
8,Royal Enfield Thunderbird X 350cc,mid
9,Royal Enfield Classic Desert Storm 500cc,mid
23,Royal Enfield Classic Chrome 500cc,mid
36,Royal Enfield Classic 350cc,mid
...,...,...
32601,Royal Enfield Classic 350cc,mid
32614,Royal Enfield Bullet Electra 350cc,mid
32633,Royal Enfield Classic 350cc,mid
32634,Royal Enfield Thunderbird 350cc,mid


In [50]:
print(df_train['brand_category'].unique())


['budget' 'mid' 'premium']


In [94]:
brand_category_map = {
    'budget': 0,
    'mid': 1,
    'premium': 2
}

# Fill brand_category_enc
df_train['brand_category_enc'] = df_train['brand_category'].map(brand_category_map)

In [95]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32648 entries, 0 to 32647
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   bike_name              32648 non-null  object 
 1   price                  32648 non-null  float64
 2   city                   32648 non-null  object 
 3   kms_driven             32648 non-null  float64
 4   owner                  32648 non-null  object 
 5   age                    32648 non-null  float64
 6   power                  32648 non-null  float64
 7   brand                  32648 non-null  object 
 8   brand_category         32648 non-null  object 
 9   brand_category_enc     32648 non-null  int64  
 10  owner_grouped          32648 non-null  object 
 11  owner_grouped_encoded  32648 non-null  int64  
dtypes: float64(4), int64(2), object(6)
memory usage: 3.0+ MB


In [96]:
df_train['brand_category'] = df_train['brand_category'].fillna('budget')

In [38]:
df_train['brand_category_enc'] = df_train['brand_category'].map(brand_category_map)

In [97]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32648 entries, 0 to 32647
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   bike_name              32648 non-null  object 
 1   price                  32648 non-null  float64
 2   city                   32648 non-null  object 
 3   kms_driven             32648 non-null  float64
 4   owner                  32648 non-null  object 
 5   age                    32648 non-null  float64
 6   power                  32648 non-null  float64
 7   brand                  32648 non-null  object 
 8   brand_category         32648 non-null  object 
 9   brand_category_enc     32648 non-null  int64  
 10  owner_grouped          32648 non-null  object 
 11  owner_grouped_encoded  32648 non-null  int64  
dtypes: float64(4), int64(2), object(6)
memory usage: 3.0+ MB


In [14]:
df_train ['brand_category_enc'] = df_train['brand_category'].map({'Budget':0, 'Mid-range':1, 'Premium':2})

In [16]:
print(df_train['owner'].unique())

['First Owner' 'Second Owner' 'Third Owner' 'Fourth Owner Or More']


In [17]:
print(df_train['owner'].value_counts())


owner
First Owner             29964
Second Owner             2564
Third Owner               108
Fourth Owner Or More       12
Name: count, dtype: int64


In [18]:
df_train['owner_grouped'] = df_train['owner'].replace({
    'Third Owner': 'Multiple Owners',
    'Fourth Owner Or More': 'Multiple Owners'
})


In [98]:
print(df_train['owner_grouped'].value_counts())

owner_grouped
First Owner        29964
Second Owner        2564
Multiple Owners      120
Name: count, dtype: int64


In [104]:
owner_map = {
    'First Owner': 0,
    'Second Owner': 1,
    'Multiple Owners': 2
}
df_train['owner_grouped_encoded'] = df_train['owner_grouped'].map(owner_map)


In [105]:
print(df_train['owner_grouped_encoded'].value_counts())

owner_grouped_encoded
0    29964
1     2564
2      120
Name: count, dtype: int64


In [106]:
print(df_train.columns)

Index(['bike_name', 'price', 'city', 'kms_driven', 'owner', 'age', 'power',
       'brand', 'brand_category', 'brand_category_enc', 'owner_grouped',
       'owner_grouped_encoded'],
      dtype='object')


In [ ]:
#df_train = pd.get_dummies(df_train, columns=['owner_grouped'], drop_first=True)


In [107]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32648 entries, 0 to 32647
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   bike_name              32648 non-null  object 
 1   price                  32648 non-null  float64
 2   city                   32648 non-null  object 
 3   kms_driven             32648 non-null  float64
 4   owner                  32648 non-null  object 
 5   age                    32648 non-null  float64
 6   power                  32648 non-null  float64
 7   brand                  32648 non-null  object 
 8   brand_category         32648 non-null  object 
 9   brand_category_enc     32648 non-null  int64  
 10  owner_grouped          32648 non-null  object 
 11  owner_grouped_encoded  32648 non-null  int64  
dtypes: float64(4), int64(2), object(6)
memory usage: 3.0+ MB


In [26]:
df_train['city'].nunique()


443

In [59]:
df_train['city'].unique()


array(['Ahmedabad', 'Delhi', 'Bangalore', 'Mumbai', 'Kalyan', 'Faridabad',
       'Mettur', 'Hyderabad', 'Kaithal', 'Gurgaon', 'Pune', 'Noida',
       'Nashik', 'Kochi', 'Allahabad', 'Samastipur', 'Nadiad', 'Lucknow',
       'Jaipur', 'Karnal', 'Gorakhpur', 'Vidisha', 'Hosur', 'Bagalkot',
       'Baripara', 'Agra', 'Dharwad', 'Vadodara', 'Jalandhar', 'Surat',
       'Chennai', 'Navi Mumbai', 'Gandhidham', 'Visakhapatnam',
       'Thrissur', 'Kolkata', 'Ernakulam', 'Barasat', 'Ghaziabad',
       'Bhubaneshwar', 'Amritsar', 'Bhopal', 'Hamirpur(hp)', 'Kottayam',
       'Arrah', 'Patiala', 'Ranga Reddy', 'Mandi', 'Ludhiana', 'Mandya',
       'Siliguri', 'Aurangabad', 'Kanpur', 'Bhilwara', 'Meerut', 'Rewari',
       'Ahmednagar', 'Wardha', 'Chandigarh', 'Ranchi', 'Panvel', 'Thane',
       'Jabalpur', 'Kota', 'Rohtak', 'Rajkot', 'Varanasi', '24 Pargana',
       'Banka', 'Nagpur', 'Banki', 'Pali', 'Chhatarpur', 'Katihar',
       'Mohali', 'Rudrapur', 'Coimbatore', 'Jajpur', 'Mysore', 'Adoni',

In [54]:
X = df_train[['age', 'kms_driven', 'power', 'brand_category_enc', 'owner_grouped_encoded']]

y= df_train['price']

In [55]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=38)

# Feature Scaling

In [110]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model Bulding

In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor




# # 🧠 Neural Network (ANN) from Keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense


In [121]:
models = {
    'Linear Regression': LinearRegression(),
    'SVM (Poly Kernel)': SVR(kernel='poly', degree=2),
    'SVM (RBF Kernel)': SVR(kernel='rbf'),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(n_jobs=-1, random_state=42),
    'XGBoost': XGBRegressor(n_jobs=-1, verbosity=0, random_state=42),

}


In [122]:
for name, model in models.items():
    model.fit(X_train, y_train)
    score = model.score(X_train, y_train)
    print(f"{name} (No Scaling): \t {score}")


Linear Regression (No Scaling): 	 0.7737293595952373
SVM (Poly Kernel) (No Scaling): 	 -0.19500854003907286
SVM (RBF Kernel) (No Scaling): 	 -0.05641914600565756
Decision Tree (No Scaling): 	 0.9995026865253644
Random Forest (No Scaling): 	 0.9938534839412573
XGBoost (No Scaling): 	 0.9925464839507483


In [115]:
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    print(name, '\t', model.score(X_train_scaled, y_train))


Linear Regression 	 0.7737293595952373
SVM (Poly Kernel) 	 0.24748785187268696
SVM (RBF Kernel) 	 -0.06488869321648161
Decision Tree 	 0.9995026865253644
Random Forest 	 0.9940386307926091
XGBoost 	 0.9925464839507483


In [125]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

xgb_model = XGBRegressor(n_jobs=-1, verbosity=0, random_state=42)
xgb_model.fit(X_train_scaled, y_train)

xgb_train_pred = xgb_model.predict(X_train_scaled)
print("XGBoost Train R² Score:", r2_score(y_train, xgb_train_pred))


XGBoost Train R² Score: 0.9925464839507483


In [62]:
X_test_scaled = scaler.transform(X_test)

In [ ]:
#without_scale_test_score
from sklearn.metrics import r2_score

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = r2_score(y_test, y_pred)
    print(f"{name} Test R² Score: {score}")


Linear Regression Test R² Score: 0.7401818666189037
SVM (Poly Kernel) Test R² Score: -0.03895633825891398
SVM (RBF Kernel) Test R² Score: -0.05611973525056535
Decision Tree Test R² Score: 0.9210429401237523
Random Forest Test R² Score: 0.9245154350235759
XGBoost Test R² Score: 0.9357836859580411


In [ ]:
#scaled_model_score
from sklearn.metrics import r2_score

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    score = r2_score(y_test, y_pred)
    print(f"{name} Test R² Score: {score}")


Linear Regression Test R² Score: 0.7401818666189031
SVM (Poly Kernel) Test R² Score: 0.23985090980893942
SVM (RBF Kernel) Test R² Score: -0.06435350384727778
Decision Tree Test R² Score: 0.9200361969791739
Random Forest Test R² Score: 0.9226262110819957
XGBoost Test R² Score: 0.9357836859580411


In [124]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_jobs=-1, verbosity=0, random_state=42)
xgb_model.fit(X_train_scaled, y_train)

xgb_pred = xgb_model.predict(X_test_scaled)
print("XGBoost Test R² Score:", r2_score(y_test, xgb_pred))


XGBoost Test R² Score: 0.9357836859580411


In [73]:
from xgboost import XGBRegressor

# Refit final model
model = XGBRegressor(n_jobs=-1, verbosity=0, random_state=42)
model.fit(X_train_scaled, y_train)

# 🔍 Print R² scores
print("✅ Final Train R² Score:", model.score(X_train_scaled, y_train))
print("📈 Final Test R² Score:", model.score(X_test_scaled, y_test))


✅ Final Train R² Score: 0.9925464839507483
📈 Final Test R² Score: 0.9357836859580411


In [75]:
from xgboost import XGBRegressor


final_model = XGBRegressor(n_jobs=-1, verbosity=0, random_state=42)
final_model.fit(X_train_scaled, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=-1,
             num_parallel_tree=None, random_state=42, ...)

In [79]:
import pickle

In [80]:
pickle.dump(final_model,open('usebike_model.pkl','wb'))

In [126]:
pickle.dump(scaler,open('scaler.pkl','wb'))